Nota generada a partir de [liga](https://www.dropbox.com/s/z465znq3wwao9ad/2.1.Un_poco_de_historia_y_generalidades.pdf?dl=0)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_local -p 8888:8888 -d palmoreck/jupyterlab:1.1.0
```

Detener el contenedor de docker:

```
docker stop jupyterlab_local
```


Documentación de la imagen de docker `palmoreck/jupyterlab:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab).

---

# Temas a considerar en un programa de máquina de alto rendimiento

Para tener un alto performance en un programa de máquina, deben considerarse las siguientes preguntas:

* ¿Qué tanto aprovecha mi programa aspectos como data reuse y data locality? 

La respuesta nos lleva a pensar en el número de instrucciones por ciclo y el número de ciclos que realiza el procesador.

* ¿Cómo es mi data layout en el almacenamiento? (forma en la que están almacenados o dispuestos los datos)

Dependiendo de la respuesta podemos elegir una arquitectura de computadoras u otra y así también un algoritmo u otro.

* ¿Cuánto data movement o data motion realiza mi programa? (flujo de datos entre los distintos niveles de jerarquía de almacenamiento o entre las máquinas en un clúster de máquinas)

La respuesta implica analizar el tráfico de datos entre las **jerarquías de almacenamiento** (o máquinas si estamos en un clúster de máquinas) y potenciales bottlenecks.

# Herramientas que tenemos a nuestra disposición para un programa de máquina de alto rendimiento

* Vectorización que pueden realizar los procesadores.

* Perfilamiento de código para lograr la eficiencia deseada. 

* Programación en lenguajes compilados en lugar de intérpretes (o combinando intérpretes con lenguajes compilados, ver [1.7.Compilar_a_C_Cython](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.7.Compilar_a_C_Cython.ipynb) ).

* Conocimiento de los propósitos con los que fueron diseñados los procesadores para explotar su capacidad. Aquí decidimos si usamos código secuencial o código en paralelo.

...además necesitamos conocer las diferentes **arquitecturas** que pueden utilizarse para cómputo en paralelo.

* ¿Alguien ya resolvió mi bottleneck?

* Experiencia en el lenguaje de programación seleccionado. 


# Un poco de historia y generalidades del sistema en una computadora

Las componentes fundamentales de tal sistema de computadora pueden simplificarse en:

* Unidades computacionales. En éstas unidades nos interesa la pregunta ¿cuántos cálculos pueden realizar por segundo? 

* Unidades de memoria. En éstas unidades nos interesa la pregunta ¿cuántos datos pueden alojar y qué tan rápido puede leerse desde y escribirse hacia las distintas jerarquías?

* Conexiones entre las unidades anteriores. Nos interesa ¿qué tan rápido pueden mover datos de un lugar a otro?

Y un dibujo simplificado y basado en una arquitectura de computadoras con nombre [Von Neumann](https://en.wikipedia.org/wiki/Von_Neumann_architecture) que nos ayuda a visualizar lo anterior en la **CPU** es el siguiente:

<img src="https://dl.dropboxusercontent.com/s/txsj5mzxyajbypa/von_Neumann.png?dl=0" heigth="500" width="500">

* La **memoria principal** es una colección de ubicaciones que almacenan datos e instrucciones. Cada ubicación consiste de una dirección (*address*) que se utiliza para accesar a la ubicación y a sus contenidos.

* La **CPU** está dividida en la unidad de control y la unidad aritmética y lógica. Aquí encontramos registers que son áreas o ubicaciones de almacenamiento (de datos, direcciones de memoria e información del estado de ejecución de un programa) de rápido acceso.

* La **interconexión** o *bus* ayuda a transferir datos e instrucciones entre la CPU y la memoria.

**Obs:** 

* En el dibujo no se está presentando los dispositivos de input y output pero sí aparecen en la arquitectura de Von Neumann.

* También no se presentan en el dibujo unidades de almacenamiento como los discos duros pero también aparecen en la arquitectura de Von Neumann. Los discos duros se consideran dentro de las unidades de memoria y la CPU se conecta a ellos mediante el *bus*.

## Jerarquías de almacenamiento

Tales jerarquías pueden visualizarse con el siguiente diagrama:

<img src="https://dl.dropboxusercontent.com/s/ahxsnpgp4rjdvw3/jerarquias_de_almacenamiento.png?dl=0" heigth="500" width="500">


**Preguntas de comprehensión**

1) ¿Qué factores han influido en que desde el 2002-2003 a la fecha, el performance de los procesadores se esté incrementando en un 20% por año vs el 50% de incremento por año que se tenía entre 1986 y 2002?

2) Menciona los componentes y realiza un esquema de una arquitectura von Neumann y descríbelas.

3) Menciona algunas de las tareas de un sistema operativo.

4) ¿Qué es un proceso y de qué consta?

5) ¿Qué es un thread?

6) ¿Qué es el threading? ¿qué ventajas nos da para la programación en un sistema de memoria compartida?

7) ¿Qué es el caché?

8) Nosotros como programadores o programadoras, ¿cómo podemos obtener ventajas del cache?

9) ¿Qué es un cache hit? ¿un cache miss?

10) De acuerdo a la taxonomía de Flynn, ¿qué tipos de arquitecturas existen? Menciona sus características, ventajas /desventajas y ejemplos.

11) Menciona algunas características y ejemplos de:

    a. sistemas de memoria distribuida.
    b. sistemas de memoria compartida.

12) ¿Qué significan los términos concurrencia, paralelo, distribuido?

13) ¿Cuáles son los enfoques que se utilizan para escribir programas en paralelo?

14) Define a cuál enfoque corresponde (de acuerdo a la pregunta 13) cada uno de los siguientes incisos:

    a) Supón que tienes 2 cores y un arreglo de tamaño 100
    
        if(rango_core módulo 2 == 0 )
            operar en los elementos 50 a 99
        else
            operar en los elementos 0 a 49
            
    Donde módulo es una operación que nos devuelve el residuo al dividir un número entre otro.
    
    b) Tenemos tres trabajadores: Aurora, Pedro, Daniel
        
        if(mi_nombre es Pedro)
            lavo el baño
        else
            voy de compras
            
15) En el cómputo en paralelo debemos realizar coordinación entre procesos o threads y considerar el load balancing. Menciona tipos de coordinación que existen y ¿a qué se refiere el load balancing?

16) ¿Cuáles son los pasos a seguir, que de acuerdo a Ian Foster, un@ puede seguir para el diseño de programas en paralelo?


**Referencias:**

1. P. Pacheco, An Introduction to Parallel Programming, Morgan Kaufmann, 2011.
